## DIA 028: Implementación de Rate limiting para Proteger la API

1. Objetivos del Día 28
Implementar Rate Limiting:

Configurar rate limiting para controlar la cantidad de solicitudes que un usuario puede realizar a la API en un período de tiempo determinado.
Proteger Endpoints Críticos:

Aplicar restricciones específicas a endpoints sensibles como /login, /register y /predict para prevenir abusos y ataques.
Mejorar la Experiencia del Usuario:

Informar adecuadamente a los usuarios cuando alcanzan el límite de solicitudes, proporcionando mensajes claros y útiles.
Actualizar Pruebas Automatizadas:

Asegurar que las políticas de rate limiting funcionan correctamente y que los usuarios son bloqueados cuando exceden los límites establecidos.
2. Configurar Rate Limiting con Flask-Limiter
Para implementar rate limiting en tu aplicación Flask, utilizaremos la extensión Flask-Limiter. Esta herramienta permite definir políticas de límite de solicitudes de manera sencilla y flexible.

2.1. Instalar Dependencias Necesarias
Añade Flask-Limiter y limits a tu archivo requirements.txt:

plaintext
Copiar
Flask-Limiter==2.8.1
limits==1.0.1
Instalar las Dependencias:

bash
Copiar
pip install -r requirements.txt
2.2. Configurar Flask-Limiter en api.py
Actualiza tu archivo api.py para configurar Flask-Limiter:

python
Copiar
# api.py

from flask_limiter import Limiter
from flask_limiter.util import get_remote_address

# Configuración de Rate Limiting
limiter = Limiter(
    key_func=get_remote_address,
    default_limits=["200 per day", "50 per hour"],
    storage_uri="memory://",  # Para producción, considera usar Redis u otro backend
    headers_enabled=True
)

limiter.init_app(app)
Explicación:

key_func=get_remote_address: Utiliza la dirección IP del cliente para aplicar las políticas de rate limiting.
default_limits: Define límites predeterminados para todas las rutas (200 solicitudes por día y 50 por hora).
storage_uri="memory://": Utiliza almacenamiento en memoria para los contadores de solicitudes. En producción, es recomendable usar un backend persistente como Redis.
headers_enabled=True: Añade encabezados HTTP que informan sobre los límites y el estado actual de las solicitudes.
3. Definir Políticas de Rate Limiting
Además de los límites predeterminados, puedes definir políticas específicas para ciertos endpoints según su sensibilidad.

3.1. Limitar Acceso al Endpoint de Login
El endpoint /login es crítico ya que podría ser objetivo de ataques de fuerza bruta. Implementaremos un límite más estricto.

python
Copiar
# api.py

@app.route('/login', methods=['POST'])
@limiter.limit("5 per minute")  # Limita a 5 intentos de login por minuto
def login():
    """
    Endpoint para que los usuarios inicien sesión y obtengan un token JWT.
    """
    data = request.get_json()
    username = data.get('username', None)
    password = data.get('password', None)
    
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400
    
    # Buscar al usuario en la base de datos
    user = User.query.filter_by(username=username).first()
    
    if not user or not bcrypt.check_password_hash(user.password, password):
        return jsonify({"msg": "Bad username or password"}), 401
    
    if not user.email_confirmed:
        return jsonify({"msg": "Por favor, confirma tu email antes de iniciar sesión."}), 403
    
    # Crear el token de acceso
    access_token = create_access_token(identity=username)
    return jsonify(access_token=access_token), 200
Explicación:

@limiter.limit("5 per minute"): Restringe el acceso al endpoint /login a 5 solicitudes por minuto desde la misma dirección IP.
3.2. Limitar Acceso al Endpoint de Registro
Para prevenir registros masivos no deseados, aplicaremos un límite al endpoint /register.

python
Copiar
# api.py

@app.route('/register', methods=['POST'])
@limiter.limit("10 per hour")  # Limita a 10 registros por hora por IP
def register():
    """
    Endpoint para registrar nuevos usuarios.
    """
    data = request.get_json()
    username = data.get('username', None)
    email = data.get('email', None)
    password = data.get('password', None)
    
    if not username or not email or not password:
        return jsonify({"msg": "Username, email and password required"}), 400
    
    # Verificar si el usuario ya existe
    if User.query.filter_by(username=username).first():
        return jsonify({"msg": "Username already exists"}), 409
    if User.query.filter_by(email=email).first():
        return jsonify({"msg": "Email already registered"}), 409
    
    # Hash de la contraseña
    hashed_password = bcrypt.generate_password_hash(password).decode('utf-8')
    
    # Crear un nuevo usuario
    new_user = User(username=username, email=email, password=hashed_password)
    db.session.add(new_user)
    db.session.commit()
    
    # Generar el token de confirmación
    token = generate_confirmation_token(email)
    
    # Enviar el email de confirmación
    confirm_url = url_for('confirm_email', token=token, _external=True)
    html = render_template('activate.html', confirm_url=confirm_url)
    subject = "Por favor confirma tu email"
    
    msg = Message(recipients=[email],
                  subject=subject,
                  html=html)
    mail.send(msg)
    
    return jsonify({"msg": "User registered successfully. Por favor, confirma tu email."}), 201
Explicación:

@limiter.limit("10 per hour"): Restringe el acceso al endpoint /register a 10 solicitudes por hora desde la misma dirección IP.
3.3. Limitar Acceso al Endpoint de Predicción
Para evitar el abuso del servicio de predicción, implementaremos un límite razonable.

python
Copiar
# api.py

@app.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("100 per day")  # Limita a 100 predicciones por día por usuario
def predict():
    """
    Endpoint para predecir el dígito a partir de una imagen.
    """
    # Código existente para manejar la predicción...
    pass  # Reemplaza con la implementación actual
Explicación:

@limiter.limit("100 per day"): Restringe el acceso al endpoint /predict a 100 solicitudes por día por usuario autenticado.
4. Aplicar Rate Limiting a Endpoints Específicos
A continuación, se muestran ejemplos de cómo aplicar rate limiting a diferentes endpoints según su criticidad.

4.1. Limitar Acceso al Endpoint de Login
Ya implementado en la sección anterior.

4.2. Limitar Acceso al Endpoint de Registro
Ya implementado en la sección anterior.

4.3. Limitar Acceso al Endpoint de Predicción
Asegúrate de aplicar el decorador @limiter.limit en el endpoint /predict como se mostró anteriormente.

python
Copiar
# api.py

@app.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("100 per day")  # Limita a 100 predicciones por día por usuario
def predict():
    """
    Endpoint para predecir el dígito a partir de una imagen.
    """
    try:
        if 'file' not in request.files:
            return jsonify({"error": "No se encontró el archivo en la solicitud."}), 400
        
        file = request.files['file']
        
        if file.filename == '':
            return jsonify({"error": "No se seleccionó ningún archivo."}), 400
        
        if not allowed_file(file.filename):
            return jsonify({"error": "Tipo de archivo no permitido. Por favor, sube una imagen PNG, JPG, JPEG o GIF."}), 400
        
        # Leer el archivo de la imagen
        img_bytes = file.read()
        cache_key = generate_cache_key(img_bytes)
        
        # Verificar si la predicción está en la cache
        cached_result = cache.get(cache_key)
        if cached_result:
            result = json.loads(cached_result)
            result['cached'] = True
            return jsonify(result), 200
        
        # Procesar la imagen
        image = Image.open(io.BytesIO(img_bytes)).convert('L')  # Convertir a escala de grises
        image = image.resize((28, 28))  # Asegurar el tamaño adecuado
        image_array = np.array(image) / 255.0  # Normalizar
        image_array = image_array.reshape(1, 28, 28, 1)  # Añadir dimensiones para el modelo
        
        # Realizar la predicción
        prediction = modelo_web.predict(image_array)
        predicted_digit = int(np.argmax(prediction, axis=1)[0])
        probability = float(np.max(prediction, axis=1)[0])
        
        result = {
            "prediccion": predicted_digit,
            "probabilidad": probability
        }
        
        # Almacenar el resultado en la cache
        cache.set(cache_key, json.dumps(result))
        result['cached'] = False
        
        return jsonify(result), 200
    
    except Exception as e:
        return jsonify({"error": str(e)}), 500
5. Actualizar el Frontend para Manejar Respuestas de Rate Limiting
Es importante que el frontend maneje adecuadamente las respuestas de rate limiting, informando al usuario cuando ha alcanzado el límite de solicitudes.

5.1. Actualizar index.html para Manejar Mensajes de Rate Limiting
Modifica el archivo templates/index.html para incluir la lógica necesaria.

html
Copiar
<!-- templates/index.html -->

<!DOCTYPE html>
<html>
<head>
    <title>Clasificador de Dígitos MNIST</title>
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
</head>
<body>

    <h1>Clasificador de Dígitos MNIST</h1>
    
    <!-- Formulario de Registro -->
    <h2>Registrar Usuario</h2>
    <form id="registerForm">
        <label for="reg_username">Usuario:</label>
        <input type="text" id="reg_username" name="username" required>
        <br>
        <label for="reg_email">Email:</label>
        <input type="email" id="reg_email" name="email" required>
        <br>
        <label for="reg_password">Contraseña:</label>
        <input type="password" id="reg_password" name="password" required>
        <br>
        <button type="submit">Registrar</button>
    </form>
    
    <div id="registerResult"></div>
    
    <!-- Formulario de Login -->
    <h2>Iniciar Sesión</h2>
    <form id="loginForm">
        <label for="username">Usuario:</label>
        <input type="text" id="username" name="username" required>
        <br>
        <label for="password">Contraseña:</label>
        <input type="password" id="password" name="password" required>
        <br>
        <button type="submit">Iniciar Sesión</button>
    </form>
    
    <div id="loginResult"></div>
    
    <!-- Formulario de Solicitud de Restablecimiento de Contraseña -->
    <h2>¿Olvidaste tu Contraseña?</h2>
    <form id="resetRequestForm">
        <label for="reset_email">Ingresa tu Email:</label>
        <input type="email" id="reset_email" name="email" required>
        <br>
        <button type="submit">Restablecer Contraseña</button>
    </form>
    
    <div id="resetRequestResult"></div>
    
    <!-- Formulario de Restablecimiento de Contraseña -->
    <h2>Restablecer Contraseña</h2>
    <form id="resetPasswordForm">
        <label for="new_password">Nueva Contraseña:</label>
        <input type="password" id="new_password" name="password" required>
        <br>
        <button type="submit">Actualizar Contraseña</button>
    </form>
    
    <div id="resetPasswordResult"></div>
    
    <!-- Sección de Administración -->
    <h2>Administración de Usuarios</h2>
    <button onclick="listUsers()">Listar Usuarios</button>
    <div id="adminUsersList"></div>
    
    <!-- Formulario de Predicción -->
    <h2>Enviar Imagen para Predicción</h2>
    <input type="file" id="fileInput" accept="image/*">
    <button onclick="uploadImage()">Enviar para Predicción</button>

    <h2>Resultado:</h2>
    <p id="result"></p>

    <script>
        let accessToken = '';

        // Manejar el formulario de registro
        $('#registerForm').submit(function(event) {
            event.preventDefault();
            const username = $('#reg_username').val();
            const email = $('#reg_email').val();
            const password = $('#reg_password').val();

            $.ajax({
                url: '/register',
                type: 'POST',
                contentType: 'application/json',
                data: JSON.stringify({ username: username, email: email, password: password }),
                success: function(response) {
                    $('#registerResult').text('Registro exitoso. Por favor, confirma tu email.');
                },
                error: function(xhr, status, error) {
                    if(xhr.status === 429){
                        $('#registerResult').text('Has alcanzado el límite de registros. Por favor, intenta más tarde.');
                    } else {
                        $('#registerResult').text('Error en el registro: ' + xhr.responseJSON.msg);
                    }
                }
            });
        });

        // Manejar el formulario de login
        $('#loginForm').submit(function(event) {
            event.preventDefault();
            const username = $('#username').val();
            const password = $('#password').val();

            $.ajax({
                url: '/login',
                type: 'POST',
                contentType: 'application/json',
                data: JSON.stringify({ username: username, password: password }),
                success: function(response) {
                    accessToken = response.access_token;
                    $('#loginResult').text('Inicio de sesión exitoso. Token obtenido.');
                },
                error: function(xhr, status, error) {
                    if(xhr.status === 429){
                        $('#loginResult').text('Has alcanzado el límite de intentos de login. Por favor, intenta más tarde.');
                    } else {
                        $('#loginResult').text('Error en el inicio de sesión: ' + xhr.responseJSON.msg);
                    }
                }
            });
        });

        // Manejar el formulario de solicitud de restablecimiento de contraseña
        $('#resetRequestForm').submit(function(event) {
            event.preventDefault();
            const email = $('#reset_email').val();

            $.ajax({
                url: '/reset_password_request',
                type: 'POST',
                contentType: 'application/json',
                data: JSON.stringify({ email: email }),
                success: function(response) {
                    $('#resetRequestResult').text(response.msg);
                },
                error: function(xhr, status, error) {
                    if(xhr.status === 429){
                        $('#resetRequestResult').text('Has alcanzado el límite de solicitudes. Por favor, intenta más tarde.');
                    } else {
                        $('#resetRequestResult').text('Error: ' + xhr.responseJSON.msg);
                    }
                }
            });
        });

        // Manejar el formulario de restablecimiento de contraseña
        $('#resetPasswordForm').submit(function(event) {
            event.preventDefault();
            const urlParams = new URLSearchParams(window.location.search);
            const token = window.location.pathname.split('/').pop();
            const new_password = $('#new_password').val();

            $.ajax({
                url: `/reset_password/${token}`,
                type: 'POST',
                contentType: 'application/json',
                data: JSON.stringify({ password: new_password }),
                success: function(response) {
                    $('#resetPasswordResult').text(response.msg);
                },
                error: function(xhr, status, error) {
                    if(xhr.status === 429){
                        $('#resetPasswordResult').text('Has alcanzado el límite de intentos. Por favor, intenta más tarde.');
                    } else {
                        $('#resetPasswordResult').text('Error: ' + xhr.responseJSON.msg);
                    }
                }
            });
        });

        // Función para listar usuarios (administradores)
        function listUsers() {
            if (!accessToken) {
                alert("Por favor, inicia sesión primero.");
                return;
            }

            $.ajax({
                url: '/admin/users',
                type: 'GET',
                headers: {
                    'Authorization': 'Bearer ' + accessToken
                },
                success: function(response) {
                    let usersHtml = '<ul>';
                    response.forEach(user => {
                        usersHtml += `<li>ID: ${user.id} | Usuario: ${user.username} | Email: ${user.email} | Confirmado: ${user.email_confirmed} | Rol: ${user.role}
                                      <button onclick="deleteUser(${user.id})">Eliminar</button></li>`;
                    });
                    usersHtml += '</ul>';
                    $('#adminUsersList').html(usersHtml);
                },
                error: function(xhr, status, error) {
                    if(xhr.status === 429){
                        $('#adminUsersList').text('Has alcanzado el límite de solicitudes. Por favor, intenta más tarde.');
                    } else {
                        $('#adminUsersList').text('Error: ' + xhr.responseJSON.msg);
                    }
                }
            });
        }

        // Función para eliminar un usuario (administradores)
        function deleteUser(userId) {
            if (!accessToken) {
                alert("Por favor, inicia sesión primero.");
                return;
            }

            if (!confirm("¿Estás seguro de que deseas eliminar este usuario?")) {
                return;
            }

            $.ajax({
                url: `/admin/users/${userId}`,
                type: 'DELETE',
                headers: {
                    'Authorization': 'Bearer ' + accessToken
                },
                success: function(response) {
                    alert(response.msg);
                    listUsers();  // Actualizar la lista de usuarios
                },
                error: function(xhr, status, error) {
                    if(xhr.status === 429){
                        alert('Has alcanzado el límite de solicitudes. Por favor, intenta más tarde.');
                    } else {
                        alert('Error: ' + xhr.responseJSON.msg);
                    }
                }
            });
        }

        function uploadImage() {
            if (!accessToken) {
                alert("Por favor, inicia sesión primero.");
                return;
            }

            var fileInput = document.getElementById('fileInput');
            var file = fileInput.files[0];
            if (!file) {
                alert("Por favor, selecciona una imagen.");
                return;
            }

            var formData = new FormData();
            formData.append('file', file);

            $.ajax({
                url: '/predict',
                type: 'POST',
                headers: {
                    'Authorization': 'Bearer ' + accessToken
                },
                data: formData,
                contentType: false,
                processData: false,
                success: function(response) {
                    if(response.error){
                        $('#result').text('Error: ' + response.error);
                    } else {
                        let cacheStatus = response.cached ? ' (Desde Cache)' : '';
                        $('#result').text('Predicción: ' + response.prediccion + ' | Probabilidad: ' + (response.probabilidad * 100).toFixed(2) + '%' + cacheStatus);
                    }
                },
                error: function(xhr, status, error) {
                    if(xhr.status === 429){
                        $('#result').text('Has alcanzado el límite de predicciones. Por favor, intenta más tarde.');
                    } else {
                        $('#result').text('Error: ' + xhr.responseJSON.error);
                    }
                }
            });
        }
    </script>

</body>
</html>
Explicación:

Manejo de Errores de Rate Limiting:
En cada solicitud AJAX, se verifica si el estado de respuesta es 429 (Too Many Requests) y se muestra un mensaje adecuado al usuario.
Actualización de Mensajes:
Mensajes informativos para usuarios que alcanzan los límites establecidos, indicando que deben esperar antes de realizar más solicitudes.
6. Actualizar las Pruebas Automatizadas para Rate Limiting
Es fundamental asegurar que las políticas de rate limiting funcionan correctamente y que los usuarios son bloqueados cuando exceden los límites establecidos.

6.1. Actualizar test_api.py para Incluir Pruebas de Rate Limiting
Añade las siguientes pruebas en tests/test_api.py:

python
Copiar
# tests/test_api.py

import pytest
from api import app, db, User, RoleEnum, generate_reset_token, confirm_reset_token
import os
from io import BytesIO
from PIL import Image
import numpy as np
import json
from flask import url_for
from unittest.mock import patch

@pytest.fixture
def client():
    app.config['TESTING'] = True
    app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///:memory:'  # Usar una base de datos en memoria para pruebas
    app.config['MAIL_SUPPRESS_SEND'] = True  # Evita el envío de emails durante las pruebas
    app.config['RATELIMIT_STORAGE_URL'] = "memory://"  # Usar almacenamiento en memoria para pruebas
    with app.test_client() as client:
        with app.app_context():
            db.create_all()
            # Crear un usuario admin para pruebas
            admin_user = User(
                username='admin',
                email='admin@example.com',
                password=bcrypt.generate_password_hash('adminpassword').decode('utf-8'),
                email_confirmed=True,
                role=RoleEnum.ADMIN
            )
            db.session.add(admin_user)
            db.session.commit()
            yield client
            db.session.remove()
            db.drop_all()

def generate_image(digit=5):
    # Genera una imagen en blanco con un dígito negro en el centro
    img = Image.new('L', (28, 28), color=255)
    return img

def register_user(client, username, email, password, role=RoleEnum.USER):
    response = client.post('/register', json={
        'username': username,
        'email': email,
        'password': password
    })
    # Actualizar el rol si es necesario
    if role != RoleEnum.USER:
        user = User.query.filter_by(username=username).first()
        user.role = role
        user.email_confirmed = True  # Confirmar email para simplificar
        db.session.commit()
    return response

def login(client, username, password):
    response = client.post('/login', json={
        'username': username,
        'password': password
    })
    if response.status_code != 200:
        return None
    data = response.get_json()
    return data.get('access_token')

@patch('api.mail.send')
def test_rate_limiting_login(mock_send, client):
    # Registrar un usuario
    register_user(client, 'testuser', 'testuser@example.com', 'testpassword')
    
    # Intentar realizar más de 5 logins por minuto
    for i in range(6):
        response = client.post('/login', json={
            'username': 'testuser',
            'password': 'wrongpassword'  # Intentos fallidos
        })
        if i < 5:
            assert response.status_code == 401
        else:
            assert response.status_code == 429  # Límite alcanzado

    # Intentar realizar un login exitoso después de alcanzar el límite
    response = client.post('/login', json={
        'username': 'testuser',
        'password': 'testpassword'
    })
    assert response.status_code == 429
    json_data = response.get_json()
    assert 'rate limit exceeded' in json_data['message'].lower()

@patch('api.mail.send')
def test_rate_limiting_register(mock_send, client):
    # Intentar registrar más de 10 usuarios por hora desde la misma IP
    for i in range(11):
        response = client.post('/register', json={
            'username': f'user{i}',
            'email': f'user{i}@example.com',
            'password': 'testpassword'
        })
        if i < 10:
            if i == 0:
                assert response.status_code == 201
            else:
                assert response.status_code in [201, 409]  # Puede fallar si el usuario ya existe
        else:
            assert response.status_code == 429  # Límite alcanzado

@patch('api.mail.send')
def test_rate_limiting_predict(mock_send, client):
    # Registrar y confirmar un usuario
    register_user(client, 'testuser', 'testuser@example.com', 'testpassword')
    
    # Iniciar sesión
    token = login(client, 'testuser', 'testpassword')
    assert token is not None
    
    # Generar una imagen para la predicción
    img = generate_image()
    img_byte_arr = BytesIO()
    img.save(img_byte_arr, format='PNG')
    img_byte_arr.seek(0)
    
    # Intentar realizar más de 100 predicciones por día
    for i in range(101):
        data = {
            'file': (img_byte_arr, 'test.png')
        }
        response = client.post('/predict', data=data, headers={'Authorization': f'Bearer {token}'}, content_type='multipart/form-data')
        if i < 100:
            # Puede ser 200 o 500 dependiendo de la implementación de la predicción
            assert response.status_code in [200, 500]
        else:
            assert response.status_code == 429  # Límite alcanzado

def test_reset_password_rate_limiting(client):
    # Registrar un usuario
    register_user(client, 'testuser', 'testuser@example.com', 'testpassword')
    
    # Intentar solicitar más de 10 restablecimientos por hora
    for i in range(11):
        response = client.post('/reset_password_request', json={'email': 'testuser@example.com'})
        if i < 10:
            assert response.status_code == 200
        else:
            assert response.status_code == 429  # Límite alcanzado

def test_admin_endpoints_rate_limiting(client):
    # Iniciar sesión como admin
    admin_token = login(client, 'admin', 'adminpassword')
    assert admin_token is not None
    
    # Intentar listar usuarios más de 200 veces por día
    for i in range(201):
        response = client.get('/admin/users', headers={'Authorization': f'Bearer {admin_token}'})
        if i < 200:
            assert response.status_code == 200
        else:
            assert response.status_code == 429  # Límite alcanzado
    
    # Intentar eliminar un usuario más de 200 veces por día
    for i in range(201):
        response = client.delete('/admin/users/1', headers={'Authorization': f'Bearer {admin_token}'})
        if i < 200:
            # Puede ser 200 o 404 si el usuario ya fue eliminado
            assert response.status_code in [200, 404]
        else:
            assert response.status_code == 429  # Límite alcanzado
Explicación:

test_rate_limiting_login:

Simula intentos fallidos de login para alcanzar el límite de 5 por minuto.
Verifica que el sexto intento retorna un error 429.
Intenta un login exitoso después del límite y verifica que también es bloqueado.
test_rate_limiting_register:

Intenta registrar 11 usuarios en una hora desde la misma IP.
Verifica que el undécimo intento es bloqueado con un error 429.
test_rate_limiting_predict:

Simula 101 solicitudes al endpoint /predict.
Verifica que el límite de 100 por día es respetado y que el intento 101 es bloqueado.
test_reset_password_rate_limiting:

Simula 11 solicitudes de restablecimiento de contraseña.
Verifica que el undécimo intento es bloqueado con un error 429.
test_admin_endpoints_rate_limiting:

Inicia sesión como administrador y realiza más de 200 solicitudes a endpoints administrativos.
Verifica que las solicitudes adicionales son bloqueadas.
7. Conclusiones y Recomendaciones
7.1. Implementación de Rate Limiting
Beneficios:

Prevención de Abusos: Limita la cantidad de solicitudes que un usuario puede realizar, reduciendo el riesgo de ataques de fuerza bruta y DoS.
Mejora de la Disponibilidad: Protege los recursos del servidor al evitar la sobrecarga causada por solicitudes excesivas.
Control de Costos: En entornos de producción donde los costos pueden estar vinculados al uso de recursos, rate limiting ayuda a mantener los costos bajo control.
Recomendaciones:

Ajustar los Límites Según Necesidad: Analiza el comportamiento de los usuarios y ajusta los límites para equilibrar entre seguridad y usabilidad.
Uso de Backends Persistentes: En producción, considera utilizar backends como Redis para almacenar los contadores de rate limiting, permitiendo escalabilidad horizontal.
Personalizar los Mensajes de Error: Proporciona mensajes claros y útiles cuando se alcanza el límite, informando al usuario cuándo podrá realizar nuevas solicitudes.
7.2. Mejorar la Experiencia del Usuario
Feedback Claro: Asegúrate de que los usuarios entiendan por qué no pueden realizar más solicitudes y cuándo podrán intentarlo nuevamente.
Opciones de Espera y Retrasos: Considera implementar retrasos exponenciales en respuestas de rate limiting para disuadir intentos de abuso.
7.3. Monitoreo y Alertas
Implementar Monitoreo: Utiliza herramientas de monitoreo para rastrear el tráfico de la API y detectar patrones inusuales que puedan indicar intentos de abuso.
Alertas Automáticas: Configura alertas para notificar al equipo de desarrollo cuando se alcanzan ciertos umbrales de solicitudes, permitiendo respuestas rápidas a incidentes.
7.4. Ampliar las Políticas de Rate Limiting
Usuarios Autenticados vs. No Autenticados: Define políticas diferentes para usuarios autenticados y no autenticados, permitiendo más solicitudes a usuarios legítimos.
Endpoints Específicos: Aplica límites personalizados a endpoints según su importancia y sensibilidad.
8. Recursos Adicionales
Flask-Limiter Documentation: Flask-Limiter Docs
Rate Limiting Best Practices: OWASP Rate Limiting
Flask Documentation: Flask Docs
Flask-JWT-Extended Documentation: Flask-JWT-Extended Docs
Flask-Mail Documentation: Flask-Mail Docs
SQLAlchemy Documentation: SQLAlchemy Docs
PyTest Documentation: PyTest Docs
itsdangerous Documentation: itsdangerous Docs
Flask Testing Documentation: Flask Testing Docs
Redis Documentation: Redis Docs
Conclusión
En el Día 28, has implementado rate limiting en tu API de predicción de dígitos MNIST, mejorando significativamente la seguridad y robustez de tu aplicación. Al limitar la cantidad de solicitudes que un usuario puede realizar en un período determinado, has protegido la API contra abusos y ataques, asegurando una mejor disponibilidad y control de recursos.

Pasos Clave Realizados:

Configuración de Flask-Limiter:
Instalaste y configuraste Flask-Limiter para gestionar las políticas de rate limiting.
Definición de Políticas Específicas:
Estableciste límites más estrictos para endpoints sensibles como /login, /register y /predict.
Aplicación de Rate Limiting:
Implementaste decoradores en los endpoints para aplicar las políticas definidas.
Actualización del Frontend:
Modificaste la interfaz para manejar adecuadamente las respuestas de rate limiting, informando a los usuarios cuando alcanzan los límites.
Actualización de Pruebas Automatizadas:
Añadiste pruebas que verifican que las políticas de rate limiting funcionan correctamente y que los usuarios son bloqueados cuando exceden los límites.
Recomendaciones para Continuar:

Monitoreo Continuo: Implementa herramientas de monitoreo para rastrear el tráfico y detectar patrones inusuales.
Ajuste de Límites: Revisa y ajusta periódicamente las políticas de rate limiting según el uso real y las necesidades de seguridad.
Expansión de Políticas: Considera implementar políticas más avanzadas, como rate limiting basado en roles o tipos de usuarios.
Integración con Otros Mecanismos de Seguridad: Complementa rate limiting con otras medidas de seguridad como firewalls de aplicaciones web (WAF) y sistemas de detección de intrusiones.